# Instalando pacotes


In [ ]:
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!sudo update-alternatives --config python3

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support
  python3.7-minimal
Suggested packages:
  python3.7-venv binfmt-support
The following NEW packages will be installed:
  libpython3.7-minimal libpython3.7-stdlib mailcap mime-support python3.7
  python3.7-minimal
0 upgraded, 6 newly installed, 0 to remove and 32 not upgraded.
Need to get 4,698 kB of archives.
After this operation, 17.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.7-minimal amd64 3.7.17-1+jammy1 [608 kB]
Get:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-minima

In [ ]:
!sudo apt-get install python3.7-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3.7-lib2to3
The following NEW packages will be installed:
  python3.7-distutils python3.7-lib2to3
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 313 kB of archives.
After this operation, 1,229 kB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-lib2to3 all 3.7.17-1+jammy1 [124 kB]
Get:2 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 python3.7-distutils all 3.7.17-1+jammy1 [189 kB]
Fetched 313 kB in 2s (200 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize fronte

In [ ]:
!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 70.7 MB/s eta 0:00:00


# Código

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS')

In [ ]:
%cd /content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS

/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS


In [ ]:
import sys
import torch
import utils
import logging
import genotypes

import argparse
import torch.nn as nn
import torch.utils
import torchvision.datasets as dset
from model import NetworkCIFAR as Network

In [ ]:
parser = argparse.ArgumentParser("cifar")
parser.add_argument('--arch', type=str, default='du_darts_c10_s0', help='which architecture to use')
parser.add_argument('--auxiliary', action='store_false', default=True, help='use auxiliary tower')
parser.add_argument('--auxiliary_weight', type=float, default=0.4, help='weight for auxiliary loss')
parser.add_argument('--batch_size', type=int, default=96, help='batch size')
parser.add_argument('--cutout', action='store_true', default=False, help='use cutout')
parser.add_argument('--cutout_length', type=int, default=16, help='cutout length')
parser.add_argument('--data', type=str, default='./dataset/', help='location of the data corpus')
parser.add_argument('--dataset', type=str, default='brainMRI', help='dataset')
parser.add_argument('--drop_path_prob', type=float, default=0.2, help='drop path probability')
parser.add_argument('--gpu_id', type=int, default=0, help='gpu device id')
parser.add_argument('--init_channels', type=int, default=36, help='num of init channels')
parser.add_argument('--layers', type=int, default=20, help='total number of layers')
parser.add_argument('--pretrained_ckpt', type=str, default=None, help='path to pretrained checkpoint')
parser.add_argument('--report_freq', type=float, default=50, help='report frequency')
parser.add_argument('--seed', type=int, default=0, help='random seed')
args = parser.parse_args(args=['--pretrained_ckpt', '/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS/log/EXP-20240206-151353-bf893620-2d6e-4b7d-8233-4f87bec9c67f/best_weights.pt'])

In [ ]:
def main():
    utils.set_seed(args.seed)
    logging.info('gpu device = %d' % args.gpu_id)
    logging.info("args = %s", args)
    print(f'gpu device = {args.gpu_id}')
    print(f'args = {args}')

    genotype = eval("genotypes.%s" % args.arch)
    # model = Network(args.init_channels, CIFAR_CLASSES, args.layers, args.auxiliary, genotype)
    model = torch.load(args.pretrained_ckpt)

    model = model.cuda()

    params = utils.count_parameters_in_MB(model)
    logging.info("param size = %f M", params)
    print(f'"param size = {params} M')

    # load pretrained weights
    # model.load_state_dict(torch.load(args.pretrained_ckpt))
    logging.info("load pretrained weights done")
    print("load pretrained weights done")

    criterion = nn.CrossEntropyLoss()
    criterion = criterion.cuda()

    if args.dataset == 'cifar10':
        train_transform, valid_transform = utils.data_transforms_cifar10(args)
        valid_data = dset.CIFAR10(root=args.data+args.dataset, train=False, download=True, transform=valid_transform)
    elif args.dataset == 'cifar100':
        train_transform, valid_transform = utils.data_transforms_cifar100(args)
        valid_data = dset.CIFAR100(root=args.data+args.dataset, train=False, download=True, transform=valid_transform)
    elif args.dataset == 'brainMRI':
        ext_list = ['.jpg', '.JPG', '.jpeg', '.png']
        root_brain = '/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS/dataset/brain_tumor_dataset'

        train_transform, valid_transform = utils.data_transforms_brainMRI(args)
        valid_data = dset.DatasetFolder(root = root_brain, loader =  utils.default_loader , extensions= ext_list, transform= valid_transform)
    else:
        raise ValueError('No Defined Dataset!!!')
    valid_queue = torch.utils.data.DataLoader(
        valid_data, batch_size=args.batch_size, shuffle=False, pin_memory=True, num_workers=2)

    with torch.no_grad():
        model.drop_path_prob = args.drop_path_prob
        valid_acc, valid_obj = infer(valid_queue, model, criterion)
        logging.info('valid_acc %f\t', valid_acc)
        print(f'valid_acc {valid_acc}')

def infer(valid_queue, model, criterion):
    objs = utils.AverageMeter()
    top1 = utils.AverageMeter()
    top5 = utils.AverageMeter()
    model.eval()

    for step, (input, target) in enumerate(valid_queue):
        input = input.cuda()
        target = target.cuda(non_blocking=True)

        logits = model(input)
        loss = criterion(logits, target)

        prec1, prec5 = utils.accuracy(logits, target, topk=(1, 2))
        n = input.size(0)
        objs.update(loss.item(), n)
        top1.update(prec1.item(), n)
        top5.update(prec5.item(), n)

        if step % args.report_freq == 0:
            logging.info('valid %03d %e %f %f', step, objs.avg, top1.avg, top5.avg)
            print(f'valid {step} {objs.avg} {top1.avg} {top5.avg}')
    return top1.avg, objs.avg


gpu device = 0
args = Namespace(arch='du_darts_c10_s0', auxiliary=True, auxiliary_weight=0.4, batch_size=96, cutout=False, cutout_length=16, data='./dataset/', dataset='brainMRI', drop_path_prob=0.2, gpu_id=0, init_channels=36, layers=20, pretrained_ckpt='/content/drive/Shareddrives/Mestrado_BarbaraMilagres/GitHub/DUDARTS/log/EXP-20240206-151353-bf893620-2d6e-4b7d-8233-4f87bec9c67f/weights.pt', report_freq=50, seed=0)
"param size = 1.928562 M
load pretrained weights done
valid 0 0.00016850912652444094 100.0 100.0
valid_acc 100.0


In [ ]:
log_format = '%(asctime)s %(message)s'
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format=log_format, datefmt='%m/%d %I:%M:%S %p')

if args.dataset == 'cifar10':
    CIFAR_CLASSES = 10
elif args.dataset == 'cifar100':
    CIFAR_CLASSES = 100
elif args.dataset == 'brainMRI':
    CIFAR_CLASSES = 2
else:
    raise ValueError('No Defined Dataset!!!')


if __name__ == '__main__':
    main()
